### Import modules and verify they work? 

In [1]:
# general python
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
import numpy as np
import os
from pathlib import Path
import yaml
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import scipy
import xarray as xr
from tqdm import tqdm
import glob
from devtools import pprint
from tqdm import tqdm

In [74]:
pip install git+https://github.com/Daafip/ewatercycle-test-parallelisation@main

  Cloning https://github.com/Daafip/ewatercycle-test-parallelisation (to revision main) to /tmp/pip-req-build-lwapcxrd
  Running command git clone --filter=blob:none --quiet https://github.com/Daafip/ewatercycle-test-parallelisation /tmp/pip-req-build-lwapcxrd
^C
ERROR: Operation cancelled by user
Note: you may need to restart the kernel to use updated packages.


#### import DA function:

In [9]:
import importlib.util

def module_from_file(module_name, file_path):
    spec = importlib.util.spec_from_file_location(module_name, file_path)
    module = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(module)
    return module

In [10]:
DA = module_from_file("DA",r'../eWaterCycle-DA/src/eWaterCycle_DA/DA.py')

In [11]:
n_particles = 3

In [12]:
ensemble = DA.Ensemble(N=n_particles)
ensemble.setup()

In [13]:
array_random_num = np.array([[np.random.random() for i in range(len(p_max_initial))] for i in range(n_particles)])
p_intial = p_min_initial + array_random_num * (p_max_initial-p_min_initial)

In [14]:
# values wihch you 
setup_kwargs_lst = []
for index in range(n_particles):
    setup_kwargs_lst.append({'parameters':','.join([str(p) for p in p_intial[index]]), 
                            'initial_storage':','.join([str(s) for s in s_0]),
                             })

In [15]:
# this initializes the models for all ensemble members. 
ensemble.initialize(model_name=["HBV"]*n_particles,
                    forcing=[camels_forcing]*n_particles,
                    setup_kwargs=setup_kwargs_lst) 

if fails to initialize, run in cmd:
[link1](https://stackoverflow.com/questions/65272764/ports-are-not-available-listen-tcp-0-0-0-0-50070-bind-an-attempt-was-made-to)
[link2](https://asheroto.medium.com/docker-error-an-attempt-was-made-to-access-a-socket-in-a-way-forbidden-by-its-access-permissions-15a444ab217b)
```bash
net stop winnat
netsh int ipv4 set dynamic tcp start=49152 num=16384
netsh int ipv6 set dynamic tcp start=49152 num=16384
net start winnat
````

In [16]:
# # #### run if fails 
# ensemble.finalize()

## Time current update step

In [22]:
%%time
ensemble.update(assimilate=False)

CPU times: user 0 ns, sys: 2.92 ms, total: 2.92 ms
Wall time: 9.7 ms


In [24]:
def H(Z):
    if len(Z) == 13:
        return Z[-1] 
    else: 
        raise SyntaxWarning(f"Length of statevector should be 9 but is {len(Z)}")

In [25]:
ensemble.initialize_da_method(ensemble_method_name = "PF", 
                              hyper_parameters = {
                                               'like_sigma_weights' : 0.05,
                                               'like_sigma_state_vector' : 0.01,
                                                 },
                              
                              state_vector_variables = "all", # the next three are keyword arguments but are needed. 
                              observation_path = ds_obs_dir,
                              observed_variable_name = "Q",
                              measurement_operator = H, 
                           
                            )

In [26]:
%%time
ensemble.update(assimilate=True)

CPU times: user 61.9 ms, sys: 5.27 ms, total: 67.2 ms
Wall time: 240 ms


### Most overhead in the assimilate step, but for other models should be useful

In [27]:
import numba

In [51]:
%%time
for ensemble_member in ensemble.ensemble_list:
    ensemble_member.update()

CPU times: user 2.87 ms, sys: 844 µs, total: 3.71 ms
Wall time: 10.9 ms


Numba not working as dependant on other code

In [52]:
# numba.__version__

In [53]:
# from numba import jit, prange

In [54]:
# @jit(nopython=False, nogil=False)
# def parallel_update(ensemble):
#     for i in prange(ensemble.N):
#         ensemble.ensemble_list[i].update()

In [55]:
# parallel_update(ensemble)

In [56]:
from dask import delayed

In [67]:
@delayed
def update_member(ensemble, i):
    ensemble.ensemble_list[i].update()
    return ensemble.ensemble_list[i].get_value("Q")

In [68]:
@delayed
def gather(*args):
    return list(args)

In [69]:
gathered = gather(*[update_member(ensemble,i) for i in range(ensemble.N)])

In [73]:
%%timeit
gathered.compute()

5.62 ms ± 216 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Run

In [28]:
# n_timesteps = int((ref_model.end_time - ref_model.start_time) /  ref_model.time_step)

# time = []
# lst_state_vector = []
# lst_Q = [] 
# for _ in tqdm(range(n_timesteps)):    
#     time.append(pd.Timestamp(ref_model.time_as_datetime.date()))
#     ensemble.update(assimilate=True)
#     lst_state_vector.append(ensemble.get_state_vector())
#     lst_Q.append(ensemble.get_value("Q").flatten())
#     # TODO: adjust so that tLag ? currently still often 3

# # end model - IMPORTANT! when working with dockers
# ensemble.finalize()

100%|███████████████████████████████████████████████████████████████████████████████| 1127/1127 [29:41<00:00,  1.58s/it]


In [29]:
# Q_m_arr = np.array(lst_Q).T
# state_vector_arr = np.array(lst_state_vector)

### process the numpy data into easily acessed data types